In [ ]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn
import seaborn as sns
from scipy import stats
from scipy.stats import norm,skew,kurtosis
import matplotlib.pyplot as plt
%matplotlib auto

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
print('train_data shape: ',train_data.shape)
print('test_data shape: ',test_data.shape)

train_data shape:  (4320, 27)
test_data shape:  (4320, 11)


In [4]:
import numpy as np
from collections import Counter

def train_data_generation(filename):
    train_file = open(filename, encoding= 'iso 8859-15')
    train_data_raw = np.loadtxt(train_file, delimiter= ',', dtype = str, skiprows = 1)

    n_parameters = 18
    n_date = len(Counter(train_data_raw[:, 0]).keys())

    train_data_raw[train_data_raw == 'NR'] = 0

    train_data_raw = train_data_raw[:, 3:].astype(np.float64)

    train_data = train_data_raw[:n_parameters, :]

    for i in range(1, n_date):
        temp = train_data_raw[(n_parameters * (i - 1)): (n_parameters * i), :]
        train_data = np.hstack((train_data, temp))

    n_series = 9
    target_row = 9

    mean_x = np.mean(train_data, axis = 1)
    std_x = np.std(train_data, axis = 1)

    for i in range(n_parameters):
        if std_x[i] != 0:
            train_data[i, :] = (train_data[i, :] - mean_x[i])/std_x[i]

    final_data = []


    for j in range(train_data.shape[1] - n_series):
        key = train_data[:, j:j+n_series].reshape(18*9, )
        value = train_data[target_row, j+n_series]
        final_data.append([key, value])

    return final_data, [mean_x, std_x]


In [5]:
train_file = r'F:\study\ml\HonyiLee2020\hw1\data\train.csv'
orgi_data, [mean_x, std_x] = train_data_generation(train_file)


In [6]:
w_shape = (1, 18*9)

#get training data set and validation set, randomly chosen.
np.random.shuffle(orgi_data)
test_data = orgi_data[:500]
train_data = orgi_data[500:]

#set initial values
w = np.zeros(w_shape, dtype = float)
b = 0
eta = 0.0006 #learning rate

n_iter = 1000 #iteration time
train_loss = 0
test_loss = 0
result = []

In [7]:
for i in range(n_iter):
    train_loss = 0
    for j in range(len(train_data)):
        x = train_data[j][0]
        y = np.dot(w, x) + b
        target = train_data[j][1]

        w_gradient = 2 * (y - target) * x.T
        b_gradient = 2 * (y - target)

        w = w - eta * w_gradient
        b = b - eta * b_gradient

        train_loss += (target - y) ** 2

    result.append(train_loss)

In [8]:
for k in range(len(test_data)):

    x = test_data[k][0]
    y = np.dot(w, x) + b
    target = test_data[k][1]

    test_loss += (target - y) ** 2

print(test_loss/5)

[16.53058956]


In [9]:
x_train = []
y_train = []
x_test = []
y_test = []

for i in train_data:
    x_train.append(i[0])
    y_train.append(i[1])

for j in test_data:
    x_test.append(j[0])
    y_test.append(j[1])


In [10]:
len(train_data)

5251

In [11]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

LR = linear_model.LinearRegression()
LR.fit(x_train, y_train)
y_pred = LR.predict(x_test)
print(mean_squared_error(y_pred, y_test))

0.14463701637196005
